Reflector Area Processing - Testing
===================================



To test the effect of manual gap filling, relevant functionality in notebook `../section-scans-full/working.org` was run for an thresholded image (`manual-M07B1-threshold.png`), and a thresholded image that was manually gap-filled (for grain reconstruction, where the interpretation of fractured grains originally being whole was on the basis of whether the gap appeared to be a fracture or not) (`manual-M07B1-threshold-redrawn.png`).

-   Note: the original analysis of these two images was performed on 21 Nov 2023 with the conclusions:
    -   Doesn't seem to be a huge difference in the distribution of grains <0.05 mm<sup>2</sup> area when reconstructing fractured grains - can still interpret this distribution as it's relatively robust.
    -   However, there are significant differences in aspect ratios, maximum sizes etc.
-   However, the discussions in `../section-scans-full/working.org` suggest that the amount of automatic gap filling may affect the observed distributions, and so further analysis is necessary.
    -   This further analysis will be performed in `further_analysis.org`.



In [1]:
FORCE_OVERWRITE = False
# Since this is testing the effect of manual gap-filling, not automatic gap filling is to be applied.
dilate_erode = False

####################################
# Don't change anything below here #
####################################
if dilate_erode:
    file_prepend = f"modified-{dilate_erode}"
else:
    file_prepend = "unmodified"

# Artificial alteration degree difference to permit comparison plotting.
alteration_degree = {"manual-M07B1-threshold":0,
                     "manual-M07B1-threshold-redrawn":1}
# Text descriptions for each level of alteration.
alteration_desc = {0:"No redraw",
                   1:"Redrawn"}

import os
def save_figure(path):
    ''' Check if an image containing figure output already exists, otherwise save that figure.

    path (string) : path to save the figure to.

    returns None
    '''
    if not os.path.exists(path) or FORCE_OVERWRITE:
        plt.savefig(path,bbox_inches="tight")
    return

import pandas as pd
import json

import sys
sys.path.insert(0,"../section-scans-full")
from util_funcs import *
from plotting import *

PLT = Plotter(alteration_degree,alteration_desc)
plot_all = PLT.plot_all

## Area Processing



Using methods detailed in `../section_scans-example/working.org` (or its derivatives), captured by the class `AreaProcessor` in `area_process.py`

Extracting area contours into .npy files and extracting areas into `areas.json`.

-   `areas.json` is a very large file without linebreaks and so shouldn't be opened with text editors.

As an update/difference to the example processing notebook, the largest grain is also added to the data in `areas.json`.



In [1]:
from area_processing import *

pix2mm = 1/1000

# Folder containing thresholded samples (stored as .png).
# Note: folder linking has been used - this is not the real path.
samples_dir = "./local_data"

# Load thresholded sample filenames from the folder.
samples = [k+".png" for k in alteration_degree]

# Check if the areas datafile needs to be regenerated on the basis of missing file or request.
# The areas datafile is specific to the processing pathway used to compute the areas (in terms of how much dilation-erosion is applied).
if not os.path.exists(file_prepend + "-areas.json") or FORCE_OVERWRITE:
    # Declare dictionary in which areas data will be stored.
    areas_data = dict()
    print("(Re)Generating areas.json ...")
    # Iterate through the samples with thresholded reflectors as identified above.
    for sample in samples:
        print(f"Looking at {sample}")
        # Initiate area processor for the active sample, conversion pixels to mm conversion factor and desired processing pathway.
        AP = AreaProcessor(os.path.join(samples_dir,sample),pix2mm,dilate_erode)
        # Retrieve contours.
        contours,larger_contours = AP.load_contours()
        # Retrieve patch areas.
        patch_areas,units = AP.find_areas()

        # Find the largest grain area before filtering.
        largest_grain = max(patch_areas)

        # Size filtering (selecting only areas larger than 5 pixels, and smaller than 0.05 mm2).
        min_reflector_area = 5 * pix2mm**2 # mm2
        max_reflector_area = 0.05 # mm2

        # Construct boolean filter based on grain size.
        size_filter = construct_minmax_filter(patch_areas,min_reflector_area,max_reflector_area)
        # Filter the patch areas using this boolean filter.
        patch_areas = patch_areas[size_filter]

        # Filter "small" and "large" contours using this boolean filter.
        contours = list_of_list_filter(contours,size_filter)
        larger_contours = list_of_list_filter(larger_contours,size_filter)

        # Check if the folder for storing filtered data in is present, and if not, create this folder.
        filtered_data_dir = "filtered_data"
        if not os.path.exists(filtered_data_dir):
            os.mkdir(filtered_data_dir)
        # Save the filtered contours if their savefiles aren't already present.
        if not os.path.exists(f"{filtered_data_dir}/{file_prepend}-{sample}"):
            np.save(f"{filtered_data_dir}/{file_prepend}-{sample}",np.array(contours,dtype=object))
            np.save(f"{filtered_data_dir}/{file_prepend}-{sample}-larger.npy",np.array(larger_contours,dtype=object))

        # Extract sample name from sample filename.
        sample = sample.replace(".png","")
        # Construct dictionary to place sample-specific area data.
        areas_data[sample] = dict()
        # Add reflector patch areas.
        areas_data[sample]["patch_areas"] = list(patch_areas)
        # Add the area considered when looking at patch areas.
        areas_data[sample]["area_studied"] = AP.area_studied()
        # Add the largest grain observed.
        areas_data[sample]["largest_grain"] = largest_grain
    # Save all samples' areas data for this processing pathway.
    with open(file_prepend + "-areas.json","w") as outfile:
        json.dump(areas_data,outfile)
else:
    print(f"Loading {file_prepend}-areas.json")
    # Load data from persistent storage.
    with open(file_prepend + "-areas.json") as infile:
        areas_data = json.load(infile)
print("... complete")

None

### Area Distribution Plotting



On the plots, the area range (x-axis) is hardcoded (to between 0 and 0.05 mm<sup>2</sup>).



In [1]:
fig = plot_all(PLT.area_distros,file_prepend,figsize=(18,8))
fig.suptitle("Area Distributions")
save_figure(os.path.join("imgs",file_prepend + "-area-distro.png"))
plt.show()

None

There's not a huge difference in the relative shapes of the distributions - the redrawn version is slightly broader.



## Reflector Area vs Nearest Neighbour Distance



On the plots, the area range (x-axis) is hardcoded (to between 0 and 0.05 mm<sup>2</sup>), and the nearest neighbour distance is hardcoded (to between 0 and 1 mm).



In [1]:
fig = plot_all(PLT.area_vs_nn_dist,file_prepend,figsize=(18,8))
fig.suptitle("Area vs Nearest Neighbour Distance")
save_figure(os.path.join("imgs",file_prepend + "-area-nn-dist.png"))
plt.show()

None

The scatter plots and histogram shapes appear quite different, but the histograms peak is in the same position.

-   Therefore the change in histogram peak position with increasing dilation-erosion in the data is likely a false effect.



## Reflector Aspect Ratios



On the plots, the aspect ratio range (x-axis) is hardcoded (to between 0 and 20).



In [1]:
fig = plot_all(PLT.aspect_ratio_distros,file_prepend,figsize=(18,8))
fig.suptitle("Aspect Ratio Distributions")
save_figure(os.path.join("imgs",file_prepend + "-aspect-ratios.png"))
plt.show()

None

Redrawing seems to have shifted average aspect ratios closer to 1 - i.e. grains more equant.

-   Note however that aspect ratios were discovered to be unsuitable for interpretation anyway so this is not a hugely relevant inference.



## Generalised Section Properties Processing



The generalised section properties (table [1](#orgadfba80)) are section-specific (as opposed to grain-specific) properties that were initially though to be useful to compare between sections.


| Property|Description|Units|
|---|---|---|
| <code>convhull</code>|area studied|mm<sup>2</sup>|
| <code>n</code>|number of reflectors considered||
| <code>total_area</code>|total area covered by reflectors|mm<sup>2</sup>|
| <code>largest</code>|area of largest reflector|mm<sup>2</sup>|
| <code>curve_fit</code>|area distribution fit parameters||
| <code>alteration</code>|quantitative alteration degree||



In [1]:
# Check if the summaries datafile needs to be regenerated on the basis of missing file or request.
if not os.path.exists(file_prepend + "-summary.csv") or FORCE_OVERWRITE:
    data = dict()
    # Iterate through samples and their area data.
    for sample,sample_area_data in areas_data.items():
        # Load patch areas.
        patch_areas = sample_area_data["patch_areas"]
        # Load area studied.
        area_studied = sample_area_data["area_studied"]
        # Load size of largest grain.
        largest_grain = sample_area_data["largest_grain"]
        # Compute distribution parameters for patch areas.
        counts,_,midpoints = bin_values(patch_areas,0.05,100)

        # Construct summary dataframe for each sample.
        data[sample] = {"convhull":area_studied, # study area
                        "n":len(patch_areas), # number of discrete reflectors after filtering
                        "total_area":sum(patch_areas), # area of reflectors after filtering
                        "largest":largest_grain, # largest continuous reflector patch area
                        "curve_fit":fit_exp_log_y(midpoints,counts)}

        # Degree of alteration assigned to each section.
        # Note: alteration_degree is imported from plotting.py
        try:
            data[sample]["alteration"] = alteration_degree[sample]
        except KeyError:
            pass

    # Convert dictionary to pandas dataframe.
    df = pd.DataFrame.from_dict(data,orient="index")
    # Save pandas dataframe to .csv file.
    df.to_csv(file_prepend + "-summary.csv")

### Comparison Plotting



After obtaining this data, comparisons can be plotted.

-   In some cases, derived parameters (that are normalised to the area studied) are more useful for comparing between sections.
    -   Reflector coverage area &rarr; reflector coverage percentage.
    -   Reflector count &rarr; reflector number density.
-   Only sections that are partially (0) or heavily (1) altered will be considered in the comparison.



In [1]:
# Force load from .csv file so that list processing is standardised.
df = pd.read_csv(file_prepend + "-summary.csv",index_col=0)
# Derived parameters that are more logical to compare between sections.
df["reflector_percentage"] = df["total_area"]/df["convhull"]
df["number_density"] = df["n"]/df["convhull"]

# Look at only sections that have an alteration index of 1 (heavy) or 0 (partly).
df = df[(df["alteration"]==1) | (df["alteration"]==0)]

curve_fits = np.array(json.loads("[" + ",".join(df["curve_fit"]) + "]"))

print("\t"*4 + " [No redraw\tManual Redraw]")
print("Largest grain /mm^2:\t\t", df["largest"].to_numpy())
print("Reflector number density /mm^-2:",df["number_density"].to_numpy())
print("Reflector coverage /%:\t\t", df["reflector_percentage"].to_numpy())
print("a/n:\t\t\t\t",curve_fits[:,0]/df["n"].to_numpy())
print("b:\t\t\t\t", curve_fits[:,1])

None

Comparing these differences to the observed differences across samples of different degrees of alteration (manually copied from the other notebook since Jupyter notebook seems unable to search for files outside of it's parent directory even if trusted):

<table>
<tr>
<td><img src="imgs/unmodified-refl-param-comparison.png"></td>
<td><img src="imgs/unmodified-area_fit_param_comp.png"></td>
</tr>
</table>

The difference between no redraw and manually redrawn seems relatively large for all metrics except the $b$ parameter in the distribution, hence differences in relative area distribution shapes should be relatively robust.

